In [ ]:
# External libraries
import numpy as np

# Drake dependencies
from pydrake.all import (
    DiagramBuilder,
    DiscreteContactApproximation,
    JacobianWrtVariable,
    LeafSystem,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
    VectorLogSink,
    MultibodyPlant,
    Context,
    OutputPort,
    RigidTransform,
    Sine,
    ConstantVectorSource
)

# Custom classes and functions
from TorqueController import TorqueController

# Helper functions
from manipulation.station import AppendDirectives, LoadScenario, MakeHardwareStation
from manipulation.utils import FindResource, RenderDiagram

In [ ]:
meshcat = StartMeshcat()

In [ ]:
def ctrl_fun(X_G_now: RigidTransform, V_G_now: np.ndarray, xy_des: np.ndarray) -> np.ndarray:
    """
    Our end effector controller for the iiwa. Taking in the current pose and
    velocity of the end effector, it calculates the desired forces.

    :param RigidTransform X_G_now: current end effector pose
    :param np.ndarray V_G_now: current end effector velocity (rdot, pdot, ydot, xdot, ydot, zdot)
    :param np.ndarray xy_des: desired x and y coordinates of end effector (x_des, y_des)
    :return F_G_des: Desired end effector torques (tau_x, tau_y, tau_z, f_x, f_y, x_z)
    """
    THETA_KP = 10
    THETA_KD = 3
    X_KP = 40
    X_KD = 20

    # TODO: Stiffness control in position, PD control in orientation
    F_DES = 0

    xyz = X_G_now.translation()
    rpy = RollPitchYaw(X_G_now.rotation()).vector()

    F_G_des = np.zeros(6)

    # Roll control (currently fixed at 0 degrees)
    F_G_des[0] = -THETA_KP * (rpy[0]) -THETA_KD * (V_G_now[0])

    # Pitch control (currently fixed at 0 degrees)
    F_G_des[1] = -THETA_KP * (rpy[1]) -THETA_KD * (V_G_now[1])

    # Yaw control (currently fixed at 0 degrees)
    F_G_des[2] = -THETA_KP * (rpy[2]) -THETA_KD * (V_G_now[2])

    # X control
    F_G_des[3] = -X_KP * (xyz[0] - xy_des[0]) -X_KD * (V_G_now[3])

    # Y control
    F_G_des[4] = -X_KP * (xyz[1] - xy_des[1]) -X_KD * (V_G_now[4])

    # Z control
    F_G_des[5] = F_DES

    return F_G_des

In [ ]:
scenario_data = """
directives:
- add_directives:
    file: package://manipulation/iiwa_and_wsg.dmd.yaml
model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_and_torque
      hand_model_name: wsg
"""

builder = DiagramBuilder()

scenario = LoadScenario(data=scenario_data)
station = builder.AddSystem(MakeHardwareStation(scenario, meshcat=meshcat))
plant = station.GetSubsystemByName("plant")

# Add our torque controller
controller = builder.AddSystem(TorqueController(plant, ctrl_fun))

controller.SetGains(Kp=0.01, Kd=0.0)

# Wire up ouputs
builder.Connect(controller.GetOutputPort("iiwa_position_command"), station.GetInputPort("iiwa.position"))
builder.Connect(controller.GetOutputPort("iiwa_torque_cmd"), station.GetInputPort("iiwa.torque"))

# Wire up inputs
builder.Connect(station.GetOutputPort("iiwa.position_measured"), controller.GetInputPort("iiwa_position_measured"))
builder.Connect(station.GetOutputPort("iiwa.velocity_estimated"), controller.GetInputPort("iiwa_velocity_measured"))

# Testing xy position control with a constant sine wave
# test = builder.AddSystem(Sine(amplitudes=[0.1,0.1], frequencies=[1.0,1.0], 
#                               phases=[0.0,1.57], is_time_based=True))

# Testing xy position control with a constant position
test = builder.AddSystem(ConstantVectorSource([0, -0.5]))

builder.Connect(test.get_output_port(0), controller.GetInputPort("xy_position_desired"))

diagram = builder.Build()
simulator = Simulator(diagram)

# Simulate
duration = 10.0
meshcat.StartRecording(set_visualizations_while_recording=False)
simulator.AdvanceTo(duration)
meshcat.PublishRecording()